# 🩺 Análise Interativa de Sintomas Cardiovasculares

Este notebook utiliza Processamento de Linguagem Natural (**spaCy**) e uma base de conhecimento para analisar frases de pacientes e sugerir um diagnóstico principal, sintoma e grau de risco.

---

## 1. Instalação e Configuração Inicial

In [1]:
# Instala as bibliotecas necessárias (Pandas e spaCy)
!pip install pandas spacy

# Baixa o modelo de linguagem em português. Isso é crucial para a similaridade.
!python -m spacy download pt_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 13.1 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


## 2. Carregamento das Bibliotecas e Modelo NLP

In [2]:
import pandas as pd
import spacy

nlp = None
# Tenta carregar o modelo de linguagem
try:
    nlp = spacy.load("pt_core_news_sm")
    print("✅ Modelo spaCy em Português carregado com sucesso!")
except OSError:
    print("❌ ERRO: Modelo 'pt_core_news_sm' não encontrado.")
    print("Certifique-se de executar o comando de download na célula anterior.")

✅ Modelo spaCy em Português carregado com sucesso!


## 3. Carregamento da Base de Conhecimento

**ATENÇÃO**: Certifique-se de que o arquivo CSV (`tabela_sintoma_diagnostico_risco.csv`) está no mesmo diretório deste notebook.

In [11]:
KNOWLEDGE_BASE_FILE = "../assets/tabela_sintoma_diagnostico_risco.csv"

def load_knowledge_base(file_path, nlp_model):
    """Carrega a base de conhecimento e pré-processa as expressões com spaCy."""
    if nlp_model is None:
        return None
    try:
        df = pd.read_csv(file_path)
        # Cria o objeto spaCy (doc) para cada expressão de primeira pessoa
        df['doc'] = list(nlp_model.pipe(df['Expressão em 1ª Pessoa']))
        return df
    except FileNotFoundError:
        print(f"❌ Erro: Arquivo da base de conhecimento não encontrado em '{file_path}'")
        return None
    except KeyError:
        print("❌ Erro: Colunas esperadas ('Expressão em 1ª Pessoa') não encontradas no CSV.")
        return None

knowledge_base_df = load_knowledge_base(KNOWLEDGE_BASE_FILE, nlp)

if knowledge_base_df is not None:
    print("✅ Base de conhecimento carregada com sucesso!")
    display(knowledge_base_df[['Expressão em 1ª Pessoa', 'Sintoma Cardiovascular', 'Grau de Risco']].head(10))

✅ Base de conhecimento carregada com sucesso!


,Expressão em 1ª Pessoa,Sintoma Cardiovascular,Grau de Risco
0,Sinto falta de ar ao deitar ou preciso usar ma...,Dispneia Paroxística Noturna (DPN) / Ortopneia,Alto Risco
1,Percebi inchaço nas pernas ou tornozelos.,Edema de Membros Inferiores,Médio Risco
2,Notei palpitações ou batimentos cardíacos irre...,Palpitações (Arritmia),Médio Risco
3,Sinto dor ou pressão no peito ao fazer esforço...,Angina de Esforço,Alto Risco
4,Já acordei à noite com falta de ar súbita.,Dispneia Paroxística Noturna (DPN),Alto Risco
5,Observei coloração azulada nos lábios ou extre...,Cianose,Alto Risco
6,"Tenho sentido tontura ou desmaios, especialmen...",Síncope / Hipotensão Postural,Alto Risco
7,Percebo pulsação forte ou visível no pescoço.,Estase Jugular,Alto Risco
8,Sinto dor ou desconforto ao inspirar profundam...,Dor Pleurítica,Médio Risco
9,Notei batimentos cardíacos muito acelerados ou...,Taquicardia / Bradicardia,Médio Risco


## 4. Função de Análise e Correspondência (NLP)

In [5]:
def analisar_sintoma_interativo(frase_usuario, base_conhecimento, nlp_model, limiar_similaridade=0.7):
    """Analisa a frase do usuário usando similaridade de vetores spaCy."""
    if base_conhecimento is None or nlp_model is None:
        print("A base de conhecimento ou o modelo NLP não foi carregado.")
        return

    # Processa a frase do usuário
    doc_usuario = nlp_model(frase_usuario.strip().lower())

    # Calcula a similaridade entre a frase do usuário e todas as expressões da base
    # (A similaridade só funciona bem se o modelo spaCy foi baixado com sucesso!)
    try:
        similarities = [doc_usuario.similarity(kb_doc) for kb_doc in base_conhecimento['doc']]
    except ValueError:
        print("❌ ERRO: A similaridade não pode ser calculada. Verifique se o modelo spaCy foi baixado.")
        return

    # Encontra a melhor correspondência
    maior_similaridade = max(similarities)
    best_match_index = similarities.index(maior_similaridade)
    
    print(f"\n--- Análise da Frase: '{frase_usuario}' ---")

    if maior_similaridade >= limiar_similaridade:
        resultado = base_conhecimento.iloc[best_match_index]
        print(f"✅ Correspondência (Sim. {maior_similaridade:.2f}):")
        print(f"   - Sintoma: {resultado['Sintoma Cardiovascular']}")
        print(f"   - Risco: {resultado['Grau de Risco']}")
        print(f"   - Diagnóstico Principal: {resultado['Diagnóstico Cardíaco Principal Associado']}")
        print(f"   - Correspondência na base: '{resultado['Expressão em 1ª Pessoa']}'")
    else:
        print(f"⚠️ Não foi possível identificar um sintoma correspondente (Sim. Máxima: {maior_similaridade:.2f}).")
        print(f"   Tente reformular a frase usando termos simples.")
    
    print("------------------------------------------")

## 5. Exemplos de Uso

Teste o sistema com diferentes frases. Tente variar a ordem das palavras e sinônimos!

In [6]:
# Exemplo 1: Dor no peito
analisar_sintoma_interativo("sinto muita dor no peito quando faço esforço", knowledge_base_df, nlp)


--- Análise da Frase: 'sinto muita dor no peito quando faço esforço' ---
✅ Correspondência (Sim. 0.71):
   - Sintoma: Angina de Esforço
   - Risco: Alto Risco
   - Diagnóstico Principal: Doença Arterial Coronariana (DAC)
   - Correspondência na base: 'Sinto dor ou pressão no peito ao fazer esforço físico.'
------------------------------------------


/tmp/ipykernel_10450/2179773610.py:13: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarities = [doc_usuario.similarity(kb_doc) for kb_doc in base_conhecimento['doc']]


In [10]:
# Exemplo 2: Edema
analisar_sintoma_interativo("minhas pernas e tornozelos estão inchados, com dor no peito", knowledge_base_df, nlp)


--- Análise da Frase: 'minhas pernas e tornozelos estão inchados, com dor no peito' ---
✅ Correspondência (Sim. 0.71):
   - Sintoma: Alteração na Parede Torácica
   - Risco: Baixo Risco
   - Diagnóstico Principal: Cardiomegalia Crônica
   - Correspondência na base: 'Observei abaulamentos ou retrações no tórax ao respirar.'
------------------------------------------


/tmp/ipykernel_10450/2179773610.py:13: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarities = [doc_usuario.similarity(kb_doc) for kb_doc in base_conhecimento['doc']]


In [8]:
# Exemplo 3: Síncope / Hipotensão
analisar_sintoma_interativo("tenho tontura ao me levantar", knowledge_base_df, nlp)


--- Análise da Frase: 'tenho tontura ao me levantar' ---
⚠️ Não foi possível identificar um sintoma correspondente (Sim. Máxima: 0.56).
   Tente reformular a frase usando termos simples.
------------------------------------------


/tmp/ipykernel_10450/2179773610.py:13: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarities = [doc_usuario.similarity(kb_doc) for kb_doc in base_conhecimento['doc']]


In [9]:
# Exemplo 4: Falta de ar deitado
analisar_sintoma_interativo("sinto que me falta o ar quando deito", knowledge_base_df, nlp)


--- Análise da Frase: 'sinto que me falta o ar quando deito' ---
⚠️ Não foi possível identificar um sintoma correspondente (Sim. Máxima: 0.56).
   Tente reformular a frase usando termos simples.
------------------------------------------


/tmp/ipykernel_10450/2179773610.py:13: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarities = [doc_usuario.similarity(kb_doc) for kb_doc in base_conhecimento['doc']]
